In [99]:
import tensorflow as tf
print(tf.__version__)
ws = Workspace.from_config()
print(ws.name, ' loaded')

2.11.0
topg  loaded


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/codes'

In [7]:
from tensorflow.keras.applications import VGG19
import os


base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')



MODEL 1 

In [21]:
from keras.preprocessing.image import ImageDataGenerator
import albumentations

def get_transforms(image_size):

    transforms_train = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        #albumentations.Lambda(image=to_uint8)
        #albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])

    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])


    return transforms_train, transforms_val

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
def load_and_preprocess_data(image_size):

    # Apply the augmentations to the train and validation sets
    
    transforms_train, transforms_val = get_transforms(image_size)
    
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=lambda x: transforms_train(image=x)['image'])
    
    test_datagen = ImageDataGenerator(rescale=1./255)


    trainGen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

    valGen = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

    testGen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=16,
   
    class_mode='binary')

    # Apply the transformations to your dataset
    return trainGen, valGen, testGen

def build_model(image_size, num_classes):
    #transfer learning
    #base_model = applications.Xception(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
    #base_model.trainable = False
    #actually deep learning connected model
    conv_base = VGG19(weights='imagenet', include_top = False, input_shape = (img_size,img_size,3))
    model = models.Sequential()
    #DONT NEED CONV LAYERS SINCE TF MODEL HAS HUGE AMOUNTS OF CONVOLUTIONAL LAYERS!!!!

    #model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
    #model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.Conv2D(512, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(1, activation='sigmoid'))
    conv_base.trainable = False

    opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),'accuracy',metrics.FalseNegatives(), metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC')])
    print("[INFO] compiled")
  
    return model


# Training and evaluating the model
def train_model(model, train_data, val_data, epochs):
    reduce_lr = ReduceLROnPlateau(monitor='val_recall_4', factor=0.5, patience=3, 
                                    verbose=1, mode='max', min_lr=0.00001)
                              
                              
    callbacks_list = [reduce_lr]

    history = model.fit(train_data,
                        epochs=epochs,
                        
                        validation_data=val_data
                        #callbacks=callbacks_list#,weigh class 1 with more importance (melanoma)
                       )

    return history

#set parameters
epochs=10
num_classes=2
image_size = 331

# Load and preprocess data
train_data, val_data, test_data = load_and_preprocess_data(image_size)
print("[INFO] creating model, transfer learning base, callbacks [off now] and compiling...")
# Create the model
model = build_model(image_size, num_classes)

print("[INFO] training model...")

# Train and evaluate the model
history = train_model(model, train_data, val_data, epochs)


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1175: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
[INFO] creating model, transfer learning base, callbacks [off now] and compiling...
[INFO] compiled
[INFO] training model...
Epoch 1/10
761/761 [==============================] - 2151s 3s/step - loss: 0.7649 - recall_4: 0.4961 - accuracy: 0.5052 - false_negatives_4: 2926.0000 - AUC: 0.5008 - PR-AUC: 0.4770 - val_loss: 0.6943 - val_recall_4: 1.0000 - val_accuracy: 0.4899 - val_false_negatives_4: 0.0000e+00 - val_AUC: 0.7175 - val_PR-AUC: 0.7180
Epoch 2/10
761/761 [==============================] - 2160s 3s/step - loss: 0.7344 - recall_4: 0.4720 - accuracy: 0.5028 - false_negatives_4: 3066.0000 - AUC: 0.4997 - PR-AUC: 0.4818 - val_loss: 0.6794 - val_recall_4: 0.0928 - val_accuracy: 0.5504 - val_false_negatives_4: 684.0000 - val_AUC: 0.6946 - val_PR-AUC: 0.7205
Epoch 3/10
761/761 [==============================] 

In [101]:
import joblib
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model_v10.pkl')

['outputs/model_v10.pkl']

In [ ]:
model.save('"/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/melanoma_vgg19.h5')

In [ ]:
for layer in conv_base.layers[15:]:
    layer.trainable = True

opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),'accuracy',metrics.FalseNegatives(), metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC')])

print("[INFO] training model...")

# Train and evaluate the model
history = train_model(model, train_data, val_data, epochs)

print("[INFO] saving model...")

# Save the model


In [25]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/melanoma_vgg19_finetune.h5')

In [28]:
val_loss,val_recall, val_acc,fn = model.evaluate(val_data)
print('val_loss:', val_loss)
print('val_recall:', val_recall)
print('val_acc:', val_acc)

97/97 [==============================] - 214s 2s/step - loss: 0.6698 - recall_4: 0.1194 - accuracy: 0.5640 - false_negatives_4: 664.0000 - AUC: 0.7054 - PR-AUC: 0.7426


ValueError: too many values to unpack (expected 4)

In [ ]:
test_loss,test_recall, test_acc,fn = model.evaluate(test_data)
print('test_loss:', test_loss)
print('test_recall:', test_recall)
print('test_acc:', test_acc)

In [32]:
from keras.models import Model
test_datagen = ImageDataGenerator(rescale=1./255)
testGen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

y_pred = model.predict(testGen)


Found 1521 images belonging to 2 classes.
96/96 [==============================] - 218s 2s/step


In [33]:
import numpy as np
y_pred_labels = np.argmax(y_pred, axis=1)
test_labels = testGen.classes
test_labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [38]:
testGen.class_indices

{'Benign': 0, 'MEL': 1}

MODEL 2 MobileNetV2 much bigger model

In [56]:
img_size=331
conv_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
conv_base.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 331, 331, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 166, 166, 32  864         ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 166, 166, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [64]:
from keras.preprocessing.image import ImageDataGenerator
import albumentations
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def get_transforms(image_size):

    transforms_train = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        #albumentations.Lambda(image=to_uint8)
        #albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])

    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])


    return transforms_train, transforms_val

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
def load_and_preprocess_data(image_size):

    # Apply the augmentations to the train and validation sets
    
    transforms_train, transforms_val = get_transforms(image_size)
    
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=lambda x: transforms_train(image=x)['image'])
    
    test_datagen = ImageDataGenerator(rescale=1./255)


    trainGen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

    valGen = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

    testGen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=16,
   
    class_mode='binary')

    # Apply the transformations to your dataset
    return trainGen, valGen, testGen

def build_model(image_size, num_classes):
    #transfer learning
    #base_model = applications.Xception(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
    #base_model.trainable = False
    #actually deep learning connected model
    conv_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
    model = models.Sequential()
    #DONT NEED CONV LAYERS SINCE TF MODEL HAS HUGE AMOUNTS OF CONVOLUTIONAL LAYERS!!!!
    #model.add(conv_base)
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(1024, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    conv_base.trainable = False

    opt = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),'accuracy',metrics.FalseNegatives(), metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC')])
    print("[INFO] compiled")
  
    return model


# Training and evaluating the model
def train_model(model, train_data, val_data, epochs):
    reduce_lr = ReduceLROnPlateau(monitor='val_recall_4', factor=0.5, patience=3, 
                                    verbose=1, mode='max', min_lr=0.00001)
                              
                              
    callbacks_list = [reduce_lr]

    history = model.fit(train_data,epochs=epochs,validation_data=val_data)
                        #callbacks=callbacks_list#,weigh class 1 with more importance (melanoma))

    return history


#set parameters
epochs=10
num_classes=2
image_size = 224

# Load and preprocess data
train_data, val_data, test_data = load_and_preprocess_data(image_size)
print("[INFO] creating model, transfer learning base, callbacks [off now] and compiling...")
# Create the model
model = build_model(image_size, num_classes)

print("[INFO] training model...")

# Train and evaluate the model
history = train_model(model, train_data, val_data, epochs)


Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
[INFO] creating model, transfer learning base, callbacks [off now] and compiling...
[INFO] compiled
[INFO] training model...
Epoch 1/10
761/761 [==============================] - 1846s 2s/step - loss: 0.7344 - recall_5: 0.4279 - accuracy: 0.5074 - false_negatives_5: 3322.0000 - AUC: 0.5011 - PR-AUC: 0.4779 - val_loss: 0.7294 - val_recall_5: 0.0027 - val_accuracy: 0.5107 - val_false_negatives_5: 752.0000 - val_AUC: 0.5007 - val_PR-AUC: 0.4927
Epoch 2/10
761/761 [==============================] - 1824s 2s/step - loss: 0.7023 - recall_5: 0.3585 - accuracy: 0.5073 - false_negatives_5: 3725.0000 - AUC: 0.4994 - PR-AUC: 0.4787 - val_loss: 0.7989 - val_recall_5: 1.0000 - val_accuracy: 0.4912 - val_false_negatives_5: 0.0000e+00 - val_AUC: 0.4446 - val_PR-AUC: 0.5041
Epoch 3/10
761/761 [==============================] 

2023-04-04 19:23:19.337513: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.42GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-04 19:23:19.594100: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-04 19:23:19.994652: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.40GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-04 19:23:20.238365: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU

In [65]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/melanoma_mobilenet_v2.h5')

In [91]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/codes'

In [93]:
from keras.models import Model
from tensorflow import keras

test_datagen = ImageDataGenerator(rescale=1./255)
testGen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

y_pred = model.predict(testGen)
y_pred_labels = np.argmax(y_pred, axis=1)
test_labels = val_data.classes
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

confusion_mtx = confusion_matrix(test_labels, y_pred_labels)

#0- melanoma 1 otherdiseases
class_names = ['Benign', 'Melanoma']

plot_confusion_matrix(conf_mat=confusion_mtx,
                      figsize=(8,8),
                      class_names=class_names,
                      show_normed=True,
                      cmap=plt.cm.Blues)

# Show the plot
plt.show()

Found 1521 images belonging to 2 classes.


In [87]:
y_pred = model.predict(testGen)
y_pred_labels = np.argmax(y_pred, axis=1)
test_labels = val_data.classes

model 3 another model

In [111]:
from keras.preprocessing.image import ImageDataGenerator
import albumentations
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def get_transforms(image_size):

    transforms_train = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        #albumentations.Lambda(image=to_uint8)
        #albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])

    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])


    return transforms_train, transforms_val

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
def load_and_preprocess_data(image_size):

    # Apply the augmentations to the train and validation sets
    
    transforms_train, transforms_val = get_transforms(image_size)
    
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=lambda x: transforms_train(image=x)['image'])
    
    test_datagen = ImageDataGenerator(rescale=1./255)


    trainGen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

    valGen = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(image_size, image_size),
    batch_size=16,
    class_mode='binary')

    testGen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(image_size, image_size),
    batch_size=16,
   
    class_mode='binary')

    # Apply the transformations to your dataset
    return trainGen, valGen, testGen

def build_model(image_size, num_classes):
    
    #actually deep learning connected model
    model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(3,3), activation='relu', input_shape=(image_size, image_size,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')])

    opt = tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),'accuracy',metrics.FalseNegatives(), metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC')])
    print("[INFO] compiled")
  
    return model


# Training and evaluating the model
def train_model(model, train_data, val_data, epochs):
    

    history = model.fit(train_data,epochs=epochs,validation_data=val_data)
                        #callbacks=callbacks_list#,weigh class 1 with more importance (melanoma))

    return history


#set parameters
epochs=10
num_classes=2
image_size = 224

# Load and preprocess data
train_data, val_data, test_data = load_and_preprocess_data(image_size)
print("[INFO] creating model, transfer learning base, callbacks [off now] and compiling...")
# Create the model
model = build_model(image_size, num_classes)

print("[INFO] training model...")

# Train and evaluate the model
history = train_model(model, train_data, val_data, epochs)


Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
[INFO] creating model, transfer learning base, callbacks [off now] and compiling...
[INFO] compiled
[INFO] training model...
Epoch 1/10
761/761 [==============================] - 1936s 3s/step - loss: 1.4373 - recall_12: 0.2504 - accuracy: 0.5090 - false_negatives_12: 4353.0000 - AUC: 0.5013 - PR-AUC: 0.4773 - val_loss: 0.6930 - val_recall_12: 0.0000e+00 - val_accuracy: 0.5101 - val_false_negatives_12: 754.0000 - val_AUC: 0.5000 - val_PR-AUC: 0.4899
Epoch 2/10
761/761 [==============================] - 1886s 2s/step - loss: 0.6995 - recall_12: 0.0329 - accuracy: 0.5200 - false_negatives_12: 5616.0000 - AUC: 0.4959 - PR-AUC: 0.4748 - val_loss: 0.6931 - val_recall_12: 0.0000e+00 - val_accuracy: 0.5101 - val_false_negatives_12: 754.0000 - val_AUC: 0.5000 - val_PR-AUC: 0.4899
Epoch 3/10
761/761 [======================

KeyboardInterrupt: 

In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
ws = Workspace.from_config()
model = Model.register(model_path='melanoma_deep_model.pkl', model_name='melanoma_deep_model', workspace=ws)
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/melanoma_deep_model_3.h5')


model 4 - Basic Model -used in thesis results


In [112]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
img_size=224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range = 0.1 ,
    horizontal_flip=False,
    channel_shift_range=.001)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
                  batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')


model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=[metrics.Recall(),'accuracy',metrics.FalseNegatives(), metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC')])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator, verbose=1)



Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/10
761/761 [==============================] - 2031s 3s/step - loss: 0.6024 - recall_13: 0.6840 - accuracy: 0.6686 - false_negatives_13: 1835.0000 - AUC: 0.7351 - PR-AUC: 0.6922 - val_loss: 0.5423 - val_recall_13: 0.7188 - val_accuracy: 0.7297 - val_false_negatives_13: 212.0000 - val_AUC: 0.8074 - val_PR-AUC: 0.7798
Epoch 2/10
761/761 [==============================] - 2093s 3s/step - loss: 0.5589 - recall_13: 0.7252 - accuracy: 0.7149 - false_negatives_13: 1596.0000 - AUC: 0.7860 - PR-AUC: 0.7370 - val_loss: 0.5577 - val_recall_13: 0.5663 - val_accuracy: 0.7167 - val_false_negatives_13: 327.0000 - val_AUC: 0.8192 - val_PR-AUC: 0.8031
Epoch 3/10
761/761 [==============================] - 2045s 3s/step - loss: 0.5326 - recall_13: 0.7413 - accuracy: 0.7321 - false_negatives_13: 1502.0000 - AUC: 0.8078 -

2023-04-05 12:26:28.896940: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [118]:
#ws = Workspace.from_config()
#model = Model.register(model_path='/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/outputs', model_name='my_model', workspace=ws)
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/basicmodel_noaug_30epoch.h5')


In [3]:
import tensorflow as tf 
model_path = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/basicmodel_noaug.h5'
model = tf.keras.models.load_model(model_path)

2023-04-06 08:13:14.294330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 08:13:41.833120: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-06 08:13:41.833229: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-06 08:13:41.833240: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [10]:
from keras.preprocessing.image import ImageDataGenerator
img_size=224

test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')
y_pred = model.predict(validation_generator)
test_labels = validation_generator.classes


Found 1539 images belonging to 2 classes.
97/97 [==============================] - 201s 2s/step


model 5 no transfer learning 

In [39]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
img_size=224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range = 0.1 ,
    horizontal_flip=False,
    channel_shift_range=.001,
    preprocessing_function=lambda x: tf.image.rgb_to_hsv(x))

test_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=lambda x: tf.image.rgb_to_hsv(x))

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
                  batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')


model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=[metrics.Recall(),'accuracy', metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC'),metrics.FalsePositives(),metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator, verbose=1)



Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/20
761/761 [==============================] - 2032s 3s/step - loss: 0.6117 - recall_2: 0.6640 - accuracy: 0.6578 - AUC: 0.7222 - PR-AUC: 0.6708 - false_positives: 2212.0000 - false_negatives_2: 1951.0000 - true_negatives: 4147.0000 - true_positives: 3856.0000 - val_loss: 0.5734 - val_recall_2: 0.8554 - val_accuracy: 0.7024 - val_AUC: 0.7875 - val_PR-AUC: 0.7656 - val_false_positives: 349.0000 - val_false_negatives_2: 109.0000 - val_true_negatives: 436.0000 - val_true_positives: 645.0000
Epoch 2/20
761/761 [==============================] - 2091s 3s/step - loss: 0.5747 - recall_2: 0.6890 - accuracy: 0.6932 - AUC: 0.7654 - PR-AUC: 0.7220 - false_positives: 1926.0000 - false_negatives_2: 1806.0000 - true_negatives: 4433.0000 - true_positives: 4001.0000 - val_loss: 0.5759 - val_recall_2: 0.7294 - val_accuracy: 0.7186 - val_AUC: 0.7955 - va

In [40]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/basicmodel_HSV_20epoch.h5')


In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras

model = keras.models.load_model('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/basicmodel_HSV_20epoch4.h5')

2023-04-07 09:54:14.817726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 09:54:23.976626: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10785 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7


model many epochs same as previous with lr adapter

In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')

img_size=224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range = 0.1 ,
    horizontal_flip=False,
    channel_shift_range=.001
    #preprocessing_function=lambda x: tf.image.rgb_to_hsv(x)
    )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
                  batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')


model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(2048, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
    learning_rate=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC'),metrics.FalsePositives(),
                       metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])
def get_lr_callback():
    lr_start   = 0.000005
    lr_max     = 0.00000125 * 8 * 8
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
    epoch=50
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator, verbose=1)



Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/25
761/761 [==============================] - 1859s 2s/step - loss: 0.6323 - recall_2: 0.5953 - precision_2: 0.6345 - accuracy: 0.6432 - AUC: 0.7019 - PR-AUC: 0.6514 - false_positives_2: 1991.0000 - false_negatives_2: 2350.0000 - true_negatives_2: 4368.0000 - true_positives_2: 3457.0000 - val_loss: 1.0345 - val_recall_2: 0.0849 - val_precision_2: 0.7901 - val_accuracy: 0.5406 - val_AUC: 0.7619 - val_PR-AUC: 0.7183 - val_false_positives_2: 17.0000 - val_false_negatives_2: 690.0000 - val_true_negatives_2: 768.0000 - val_true_positives_2: 64.0000
Epoch 2/25
761/761 [==============================] - 1922s 3s/step - loss: 0.6061 - recall_2: 0.6284 - precision_2: 0.6734 - accuracy: 0.6771 - AUC: 0.7400 - PR-AUC: 0.6919 - false_positives_2: 1770.0000 - false_negatives_2: 2158.0000 - true_negatives_2: 4589.0000 - true_positives_2: 3649.0000 - v

In [17]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/gputrainm8/code/Users/Miguel.Marinho.Silva/models/basicmodel_NOHSV_25epoch_lr1e-4.h5')


In [19]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute
ws = Workspace.from_config()
compute_target = ws.compute_targets['gputrainm8']

compute_target
compute_target.stop(wait_for_completion=True)

model 7 neural network of cnns 

In [ ]:
from tensorflow import keras
halfNet = keras.models.load_model('Half-U-Net.tf/')
vgg19 = keras.models.load_model('VGG19-color')
resnet = keras.models.load_model('InceptionResNetV2')
nasnet = keras.models.load_model('NASNetLarge')
MobileNetV2 = keras.models.load_model('MobileNetV2')

In [8]:
from tensorflow.keras import metrics
"""
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range = 0.1 ,
    horizontal_flip=False,
    channel_shift_range=.001)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
                  batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')
"""
opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=[metrics.Recall(),'accuracy',metrics.FalseNegatives(), metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC')])

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
# stop when no longer improving 15 epochs and restore weights of the best model.
early_stop=EarlyStopping(patience=5, restore_best_weights=True)

In [10]:
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator, callbacks=early_stop, verbose=1)


2023-04-04 10:27:27.477415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8204
2023-04-04 10:27:49.692829: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-04 10:28:16.168116: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55ba08f81c60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-04 10:28:16.168151: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2023-04-04 10:28:16.770710: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-04 10:28:19.057572: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-04 10:28:19.122216: I tensorflow/compiler/jit/xla_compilation_cache.cc:47

 37/761 [>.............................] - ETA: 33:10 - loss: 0.0000e+00 - recall: 0.5338 - accuracy: 0.5236 - false_negatives: 131.0000 - AUC: 0.5517 - PR-AUC: 0.5001

KeyboardInterrupt: 